# GEB.CL Price Analysis

This notebook loads `GEB.CL` from the SQLite database and runs the same analysis set (price, volume, RSI, and Fibonacci by timeframe).

In [1]:
import sqlite3
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
TICKER = "GEB.CL"
candidate_paths = [Path("stock_data.db"), Path("../stock_data.db")]
db_path = next((p for p in candidate_paths if p.exists()), None)

if db_path is None:
    raise FileNotFoundError("Could not find stock_data.db in current or parent directory.")

db_path

PosixPath('../stock_data.db')

In [3]:
query = """
SELECT date, open, high, low, close, adj_close, volume, dividends, stock_splits
FROM ticker_prices
WHERE ticker = ?
ORDER BY date
"""

with sqlite3.connect(db_path) as conn:
    df = pd.read_sql_query(query, conn, params=(TICKER,), parse_dates=["date"])

if df.empty:
    raise ValueError(f"No rows found for {TICKER}.")

# Filter to 2018 and later
df = df[df["date"] >= "2018-01-01"].reset_index(drop=True)

df.head()


,date,open,high,low,close,adj_close,volume,dividends,stock_splits
0,2018-01-01,2015.0,2015.0,2015.0,2015.0,1072.724121,0.0,0.0,0.0
1,2018-01-02,2010.0,2020.0,2010.0,2010.0,1070.062378,136016.0,0.0,0.0
2,2018-01-03,2020.0,2025.0,2015.0,2020.0,1075.386108,425874.0,0.0,0.0
3,2018-01-04,2020.0,2020.0,2020.0,2020.0,1075.386108,0.0,0.0,0.0
4,2018-01-05,2020.0,2025.0,2020.0,2020.0,1075.386108,276754.0,0.0,0.0


In [4]:
df["close_20ma"] = df["close"].rolling(20).mean()
df["close_50ma"] = df["close"].rolling(50).mean()
df["close_200ma"] = df["close"].rolling(200).mean()

# Exponential Moving Averages
df["ema_20"] = df["close"].ewm(span=20, adjust=False).mean()
df["ema_50"] = df["close"].ewm(span=50, adjust=False).mean()

# Bollinger Bands (20-day, 2 std)
_bb_std = df["close"].rolling(20).std()
df["bb_upper"] = df["close_20ma"] + 2 * _bb_std
df["bb_lower"] = df["close_20ma"] - 2 * _bb_std


In [5]:
# Interactive Plotly chart (Price + Volume + MAs)
fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line={'width': 2}), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['close_20ma'], name='20-day MA', line={'width': 1.5, 'dash': 'dot', 'color': 'gold'}), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['close_50ma'], name='50-day MA', line={'width': 1.5, 'color': 'orange'}), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['close_200ma'], name='200-day MA', line={'width': 1.5, 'color': 'red'}), secondary_y=False)

fig.add_trace(go.Bar(x=df['date'], y=df['volume'], name='Volume', opacity=0.25), secondary_y=True)

fig.update_layout(
    title=f'{TICKER} Close Price, Volume & Moving Averages (Plotly)',
    xaxis_title='Date',
    yaxis_title='Price',
    legend={'orientation': 'h', 'yanchor': 'bottom', 'y': 1.02, 'xanchor': 'left', 'x': 0},
    bargap=0,
    template='plotly_dark',
)
fig.update_yaxes(title_text='Volume', secondary_y=True)
fig.show()


In [6]:
# RSI calculation (7, 14 and 21)
delta = df['close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)

# RSI 7 — fastest, best for short-term signals
avg_gain_7 = gain.ewm(alpha=1/7, min_periods=7, adjust=False).mean()
avg_loss_7 = loss.ewm(alpha=1/7, min_periods=7, adjust=False).mean()
df['rsi_7'] = 100 - (100 / (1 + avg_gain_7 / avg_loss_7))

# RSI 14 — standard medium-term
avg_gain_14 = gain.ewm(alpha=1/14, min_periods=14, adjust=False).mean()
avg_loss_14 = loss.ewm(alpha=1/14, min_periods=14, adjust=False).mean()
df['rsi_14'] = 100 - (100 / (1 + avg_gain_14 / avg_loss_14))

# RSI 21 — slower, better for position/swing confirmation
avg_gain_21 = gain.ewm(alpha=1/21, min_periods=21, adjust=False).mean()
avg_loss_21 = loss.ewm(alpha=1/21, min_periods=21, adjust=False).mean()
df['rsi_21'] = 100 - (100 / (1 + avg_gain_21 / avg_loss_21))


In [7]:
# Interactive Plotly chart (RSI 7, 14 & 21)
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['date'], y=df['rsi_7'],  name='RSI 7',  line={'color': '#00CC44', 'width': 1.5, 'dash': 'dot'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['rsi_14'], name='RSI 14', line={'color': 'orange',  'width': 1.5}))
fig.add_trace(go.Scatter(x=df['date'], y=df['rsi_21'], name='RSI 21', line={'color': '#FF1111', 'width': 1.5, 'dash': 'dash'}))
fig.add_hline(y=70, line_dash='dash', line_color='red',    annotation_text='Overbought 70', annotation_position='left')
fig.add_hline(y=30, line_dash='dash', line_color='#4444FF', annotation_text='Oversold 30',   annotation_position='left')
fig.update_layout(
    title=f'{TICKER} RSI 7, 14 & 21 (Plotly)',
    xaxis_title='Date',
    yaxis_title='RSI',
    legend={'orientation': 'h', 'yanchor': 'bottom', 'y': 1.02, 'xanchor': 'left', 'x': 0},
    template='plotly_dark',
)
fig.update_yaxes(range=[0, 100])
fig.show()


In [8]:
# Interactive Plotly chart — EMA 20 & EMA 50
# EMAs react faster than simple MAs; useful for short/medium-term signals.
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line={'width': 1.5, 'color': 'steelblue'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['ema_20'], name='EMA 20', line={'width': 2.5, 'color': '#00CC44'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['ema_50'], name='EMA 50', line={'width': 2.5, 'color': '#FF1111'}))

fig.update_layout(
    title=f'{TICKER} — Exponential Moving Averages (EMA 20 & 50)',
    xaxis_title='Date',
    yaxis_title='Price',
    legend={'orientation': 'h', 'yanchor': 'bottom', 'y': 1.02, 'xanchor': 'left', 'x': 0},
    template='plotly_dark',
)
fig.show()


In [9]:
# Interactive Plotly chart — Bollinger Bands (20-day, ±2 std)
# Upper/Lower bands identify overbought/oversold zones based on volatility.
fig = go.Figure()

# Shaded band (upper → lower fill)
fig.add_trace(go.Scatter(
    x=pd.concat([df['date'], df['date'][::-1]]),
    y=pd.concat([df['bb_upper'], df['bb_lower'][::-1]]),
    fill='toself',
    fillcolor='rgba(180, 180, 255, 0.18)',
    line={'color': 'rgba(0,0,0,0)'},
    name='BB Band',
    showlegend=True,
))

fig.add_trace(go.Scatter(x=df['date'], y=df['bb_upper'], name='BB Upper', line={'width': 2.5, 'color': '#FF1111'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['close_20ma'], name='20-day MA (Mid)', line={'width': 2, 'dash': 'dot', 'color': '#FFD700'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['bb_lower'], name='BB Lower', line={'width': 2.5, 'color': '#00CC44'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line={'width': 1.5, 'color': 'white'}))

fig.update_layout(
    title=f'{TICKER} — Bollinger Bands (20-day, ±2 std)',
    xaxis_title='Date',
    yaxis_title='Price',
    legend={'orientation': 'h', 'yanchor': 'bottom', 'y': 1.02, 'xanchor': 'left', 'x': 0},
    template='plotly_dark',
)
fig.show()


In [10]:
# Fibonacci retracement levels (auto swing detection on recent window)
fib_lookback = 252  # roughly 1 trading year
swing_df = df.tail(fib_lookback).copy()

if swing_df.empty:
    raise ValueError('Not enough data to compute Fibonacci levels.')

swing_high = float(swing_df['high'].max())
swing_low = float(swing_df['low'].min())
high_idx = swing_df['high'].idxmax()
low_idx = swing_df['low'].idxmin()
trend = 'uptrend' if low_idx < high_idx else 'downtrend'

fib_ratios = [0.0, 0.236, 0.382, 0.5, 0.618, 0.786, 1.0]
if trend == 'uptrend':
    fib_levels = {f'{int(r*100)}%': swing_high - (swing_high - swing_low) * r for r in fib_ratios}
else:
    fib_levels = {f'{int(r*100)}%': swing_low + (swing_high - swing_low) * r for r in fib_ratios}

fib_table = pd.DataFrame({'level': list(fib_levels.keys()), 'price': list(fib_levels.values())})
fib_table['price'] = fib_table['price'].round(2)
print(f'Swing range: {swing_low:.2f} to {swing_high:.2f} ({trend})')
fib_table

Swing range: 2680.00 to 3395.00 (uptrend)


,level,price
0,0%,3395.00
1,23%,3226.26
2,38%,3121.87
3,50%,3037.50
4,61%,2953.13
5,78%,2833.01
6,100%,2680.00


In [11]:
# Interactive Plotly chart — Fibonacci retracement (black theme)
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Candlestick(
        x=swing_df['date'],
        open=swing_df['open'],
        high=swing_df['high'],
        low=swing_df['low'],
        close=swing_df['close'],
        name='Price'
    )
)

for label, value in fib_levels.items():
    fig.add_hline(
        y=value,
        line_width=1,
        line_dash='dot',
        annotation_text=f'Fib {label}: {value:.2f}',
        annotation_position='right'
    )

fig.update_layout(
    title=f'{TICKER} Fibonacci Retracement ({fib_lookback} sessions)',
    xaxis_title='Date',
    yaxis_title='Price',
    template='plotly_dark',
    paper_bgcolor='black',
    plot_bgcolor='black',
    font=dict(color='white'),
    xaxis_rangeslider_visible=False,
    margin=dict(r=160),
)
fig.show()


In [12]:
# ── Position Trade Fibonacci (~90 trading days, ~4.5 months) ───────────────
fib_lookback_position = 90
swing_df_pos = df.tail(fib_lookback_position).copy()

swing_high_p = float(swing_df_pos['high'].max())
swing_low_p  = float(swing_df_pos['low'].min())
high_idx_p   = swing_df_pos['high'].idxmax()
low_idx_p    = swing_df_pos['low'].idxmin()
trend_p      = 'uptrend' if low_idx_p < high_idx_p else 'downtrend'

fib_ratios = [0.0, 0.236, 0.382, 0.5, 0.618, 0.786, 1.0]
if trend_p == 'uptrend':
    fib_levels_p = {f'{int(r*100)}%': swing_high_p - (swing_high_p - swing_low_p) * r for r in fib_ratios}
else:
    fib_levels_p = {f'{int(r*100)}%': swing_low_p + (swing_high_p - swing_low_p) * r for r in fib_ratios}

print(f'[Position] Swing range: {swing_low_p:.2f} → {swing_high_p:.2f} ({trend_p})')

fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=swing_df_pos['date'],
    open=swing_df_pos['open'], high=swing_df_pos['high'],
    low=swing_df_pos['low'],  close=swing_df_pos['close'],
    name='Price'
))
for label, value in fib_levels_p.items():
    fig.add_hline(y=value, line_width=1, line_dash='dot',
                  annotation_text=f'Fib {label}: {value:.2f}',
                  annotation_position='right')
fig.update_layout(
    title=f'{TICKER} Fibonacci — Position Trade ({fib_lookback_position} sessions, ~4.5 months)',
    xaxis_title='Date', yaxis_title='Price',
    template='plotly_dark',
    paper_bgcolor='black', plot_bgcolor='black',
    font=dict(color='white'),
    xaxis_rangeslider_visible=False,
    margin=dict(r=160),
)
fig.show()


[Position] Swing range: 2830.00 → 3395.00 (downtrend)


In [13]:
# ── Monthly Swing Fibonacci (~22 trading days) ─────────────────────────────
fib_lookback_month = 22
swing_df_month = df.tail(fib_lookback_month).copy()

swing_high_m = float(swing_df_month['high'].max())
swing_low_m  = float(swing_df_month['low'].min())
high_idx_m   = swing_df_month['high'].idxmax()
low_idx_m    = swing_df_month['low'].idxmin()
trend_m      = 'uptrend' if low_idx_m < high_idx_m else 'downtrend'

fib_ratios = [0.0, 0.236, 0.382, 0.5, 0.618, 0.786, 1.0]
if trend_m == 'uptrend':
    fib_levels_m = {f'{int(r*100)}%': swing_high_m - (swing_high_m - swing_low_m) * r for r in fib_ratios}
else:
    fib_levels_m = {f'{int(r*100)}%': swing_low_m + (swing_high_m - swing_low_m) * r for r in fib_ratios}

print(f'[Monthly] Swing range: {swing_low_m:.2f} → {swing_high_m:.2f} ({trend_m})')

fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=swing_df_month['date'],
    open=swing_df_month['open'], high=swing_df_month['high'],
    low=swing_df_month['low'],  close=swing_df_month['close'],
    name='Price'
))
for label, value in fib_levels_m.items():
    fig.add_hline(y=value, line_width=1, line_dash='dot',
                  annotation_text=f'Fib {label}: {value:.2f}',
                  annotation_position='right')
fig.update_layout(
    title=f'{TICKER} Fibonacci — Monthly Swing ({fib_lookback_month} sessions)',
    xaxis_title='Date', yaxis_title='Price',
    template='plotly_dark',
    paper_bgcolor='black', plot_bgcolor='black',
    font=dict(color='white'),
    xaxis_rangeslider_visible=False,
    margin=dict(r=160),
)
fig.show()


[Monthly] Swing range: 2830.00 → 3395.00 (downtrend)


In [14]:
# ── Weekly Swing Fibonacci (~10 trading days) ──────────────────────────────
fib_lookback_week = 10
swing_df_week = df.tail(fib_lookback_week).copy()

swing_high_w = float(swing_df_week['high'].max())
swing_low_w  = float(swing_df_week['low'].min())
high_idx_w   = swing_df_week['high'].idxmax()
low_idx_w    = swing_df_week['low'].idxmin()
trend_w      = 'uptrend' if low_idx_w < high_idx_w else 'downtrend'

if trend_w == 'uptrend':
    fib_levels_w = {f'{int(r*100)}%': swing_high_w - (swing_high_w - swing_low_w) * r for r in fib_ratios}
else:
    fib_levels_w = {f'{int(r*100)}%': swing_low_w + (swing_high_w - swing_low_w) * r for r in fib_ratios}

print(f'[Weekly] Swing range: {swing_low_w:.2f} → {swing_high_w:.2f} ({trend_w})')

fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=swing_df_week['date'],
    open=swing_df_week['open'], high=swing_df_week['high'],
    low=swing_df_week['low'],  close=swing_df_week['close'],
    name='Price'
))
for label, value in fib_levels_w.items():
    fig.add_hline(y=value, line_width=1, line_dash='dot',
                  annotation_text=f'Fib {label}: {value:.2f}',
                  annotation_position='right')
fig.update_layout(
    title=f'{TICKER} Fibonacci — Weekly Swing ({fib_lookback_week} sessions)',
    xaxis_title='Date', yaxis_title='Price',
    template='plotly_dark',
    paper_bgcolor='black', plot_bgcolor='black',
    font=dict(color='white'),
    xaxis_rangeslider_visible=False,
    margin=dict(r=160),
)
fig.show()


[Weekly] Swing range: 2915.00 → 3050.00 (uptrend)
